# ** Notebook: Named Entity Recognition in Wolof**

La reconnaissance des entités nommées (NER) est une tâche en traitement naturel du langage qui consiste à identifier les mots importants dans un texte. Il peut s'agir des mots correspondant à des personnes, des villes, des évènements, etc. Ce sont ces mots qu'on appelle des entités nommées.

Dans ce notebook, l'objectif est de créer un modèle capable de reconnaître de telles entités dans du texte en wolof.

Notebook réalisé par [Alimoudine IDRISSOU](https://www.linkedin.com/in/ialim/), de l'équipe BylyAI

Dans la reconnaissance d'entité nommées , il existe plusieurs modèles de base apprécier dans la tache du ner tels que lstm, bert, xlm roberta, xlmr , spacy, flair.Mais pour les langues africains la performances de ses modèles reste minime car ces modèles sont pas préentrainés pour les langues africaines.
Nous avons expérimentés la majorité de ces modèles ...
* **LSTM**
L'expérimentation du modèle lstm bidirectionnel nous à donné une très bonne performe (72% lorsqu'il s'agit de l'évaluation sur le val set. Mais une performance minable  sur le test set (39% pour la moyenne) .Ce qui montre que ce modèle nécessite encore plus de ressources d'entrainements pour avoir une bonne performance.
* **bert**
 Le modèle bert étant pré-entrainer sur de plus large data set nous à donner un score f1 avoisinnant 63%. Ce qui est pluot mieux avec des epocs maximales de 20.Ce qui montre qu'on aurait plus de performance avec plus d'epocs ...
* **roberta**
Le modèle roberta base nous à donner un score de plutot 64-65% qui est plus impressionnant avec seuleument 10 epocs.
* **xlmroberta**
Le modèle xlmr était pour le moment le modèle plus performant nous ayant permis d'avoir le plus large score de 67% avec seuleument 10 epocs.Ce modèle étant un modèle multilingue nous avions donc éssayer de tester ce modèle sur des ensembles de langues avec la langue wolof puis choisir les plus performant set de langue avec la langue wolof.Nous avions remarqués que le swa data set et le wol data sont de beaux paires  car faisant partir  de la meme  famille de langue mais aussi au vue des resulats obtenus.Il en ait de meme pour le yoruba.
Le modèle xlmroberta est un très bon choix par rapport aux autres modèles.
* **afroxlmr-large-ner-masakhaner-1.0_2.0.**
Ce modèle est celui qui jusque là nous à donner le plus bon résulat.Contrairement aux autres modèles, ce modèle à la particularité d'etre entrainer sur les langues africaines.Nous l'avons expérimentés avec des features et notre score semble mieux que celui obtenue avec le modèle xlmr.Dans ce notebook nous avons fine tuner ce modèle avec un F1 score est de 84% en moyenne.

# Nous installons ici les packages et dépendances nécessaires pour notre environnement

In [1]:
# # Install packages
!pip install seqeval
!pip install jedi
!pip install datasets
!pip install -U transformers
!pip install -U accelerate
!pip3 install pandas python_crfsuite summarytools sklearn_crfsuite unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=34fe17e9ea0f913f5d8c1bae5ce731c52d1204ac04dacc530de1c8e7f618ac3b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s 

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:

def read_and_parse(counter=0, filepath ="/kaggle/input/nerafrican/wol/train.txt"):
  dataset = []
  with open(filepath, encoding="utf-8") as f:
    for line in f:
      temp = []
      line = line.strip()
      if len(line) < 2  or line == "\n":
        counter +=1
      else:
        sentence_id = counter
        temp.append(sentence_id)
        temp += line.split(" ")
        dataset.append(temp)
  df = pd.DataFrame(dataset, columns =['sentence_id', 'words', 'labels'], dtype = str)
  return df, counter


In [5]:
train_data, count = read_and_parse(0, "/kaggle/input/nerafrican/wol/train.txt")
val_data, count = read_and_parse(count, "/kaggle/input/nerafrican/wol/dev.txt")




In [6]:
# Tags in the dataset
original_tags = list(train_data['labels'].value_counts().keys())
num_tags = len(original_tags)

print("La liste des tag dans notre jeu de données train:", original_tags)

print("\nLe nombre de tag dans notre jeu de données:", len(original_tags))

La liste des tag dans notre jeu de données train: ['O', 'B-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-DATE', 'B-DATE', 'I-ORG', 'I-LOC']

Le nombre de tag dans notre jeu de données: 9


In [7]:
# Aggregate lists of words into sentences
train_sentences, train_tags = [], []
sent, labels = "", []
for index, row in train_data.iterrows():
    sent += str(row.words) + " "
    labels.append(str(row.labels))
    if row.words in [".", "?", "!"]:
        if len(labels) > 1:
            train_sentences.append(sent)
            train_tags.append(labels)
        sent, labels = "", []

val_sentences, val_tags = [], []
sent, labels = "", []
for index, row in val_data.iterrows():
    sent += str(row.words) + " "
    labels.append(str(row.labels))
    if row.words in [".", "?", "!"]:
        if len(labels) > 1:
            val_sentences.append(sent)
            val_tags.append(labels)
        sent, labels = "", []



In [8]:
# some stats
print("Le nombre total de phrases dans le train dataset est {:,}:".format(len(train_sentences)))
print("Le nombre total de mots dans le train dataset est {:,}:".format(train_data.shape[0]))
print("Le nombre total de phrases dans le val dataset est {:,}:".format(len(val_sentences)))
print("Le nombre total de mots dans le val dataset est {:,}:".format(val_data.shape[0]))


Le nombre total de phrases dans le train dataset est 1,801:
Le nombre total de mots dans le train dataset est 36,805:
Le nombre total de phrases dans le val dataset est 259:
Le nombre total de mots dans le val dataset est 4,384:


# Statistiques Générales sur les données de train et de val

In [9]:
#statistics on the dataset
from summarytools import dfSummary, tabset


# Assuming you have dataframes like pd_train_data, pd_augment_ner_iob_data, and pd_dev_data

# Create summaries for your dataframes
train_summary = dfSummary(train_data)
val_summary = dfSummary(val_data)


# Render the summaries to HTML
train_html = train_summary.to_html()
val_html = val_summary.to_html()


# Create a tabset
tabset({
    "train_data": train_html,
    "val_data": val_html,

})


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,sentence_id[object],1. 15792. 18473. 11334. 6545. 17966. 11857. 6238. 14849. 174210. 60911. other,"83 (0.2%)81 (0.2%)74 (0.2%)74 (0.2%)72 (0.2%)72 (0.2%)71 (0.2%)70 (0.2%)65 (0.2%)65 (0.2%)36,078 (98.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAADaElEQVR4nO3cQWoUYRRG0VehiWbSTQhkCbqHLMLFuohehHsIITUwMZNyIIo41lvQnjPK8B9ceDTUl2XbtoHC1d4P4P8hNjJiIyM2MmIjIzYyYiMjNjJiIyM2MlfLsnxcluX93g/h8l3NzIeZebf3Q7h8zigZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkbmamS8z823vh3D5Fv/FiIozSkZsZKyryFhXkXFGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIGLyQMXgh44ySERsZgxcyBi9knFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERkZsZKyryFhXkXFGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i44ySERsZgxcyBi9knFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeRsa4i44ySERsZ6yoy1lVknFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsg4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMs4oGbGRERsZ6yoy1lVknFEyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjLWVWSsq8g4o2TERsbghYzBCxlnlIzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMs4omWVmbmbmeu+HcLHetm17mZk53N3dfToej7d7v4jLtK7r07Isn7dtezkcj8fbh4eHl9Pp9Lr3w7gsz8/P78/n8+3j4+P1zLwcZmZOp9Pr/f39153fxmW6+fmHHwhkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkDjM/PgXZ+yFcnj+7Oqzr+nQ+n2/nt09B4G9Z1/VpZt5mfBbOv/frs3DrKjJ+IJARGxmxkREbGbGRERsZsZERGxmxkfkOCvhuJDOJMoEAAAAASUVORK5CYII="">",0(0.0%)
2,words[object],"1. ,2. .3. ci4. -5. a6. yi7. ko8. ak9. bi10. la11. other","2,810 (7.6%)1,604 (4.4%)1,141 (3.1%)688 (1.9%)502 (1.4%)480 (1.3%)470 (1.3%)455 (1.2%)445 (1.2%)361 (1.0%)27,849 (75.7%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAAERElEQVR4nO3dQUojaQCG4V8JTrtJCILH8AA5xBx2DpEDeA8Ra9FxeuMsukecaW2VSb0JmefZCQYK+qVCFf/Xnj09PQ0onB/6Avj/EBsZsZERGxmxkREbGbGRERsZsZE5G2NcjjEuPvGZb09PT7uZrocTtri6uvp9uVyuP/qBaZruz87O/hAcn7VYLpfrzWazW61Wj+/98sPDw5ftdru+u7u7GGOIjU9ZjDHGarV6vL6+/vrBz1zOeD2cMA8IZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGZjHG93NqH/nlj/4evGYxTdP9drtdjw+eU5um6X6M8W3ey+IU2SCQOfNfZlF5787mLsbe/HJdZUnFPr25rrKkYt/eW1dZUrE3XuqSERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGReXPwYtzCvv1y8GLcwj45Fk7G4IXMW3c2dzT27tXBi6ELc/hp8GLowlzeGrwYurB3XuqSERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGR+WnwYujCXF4dvBi6MAfHwskYvJDxgEDm31+jvj6ZzT/WVVZVzOn873XVZrPZ/YjuM38OEj7seV3142erKmbjAYGM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3Mm39OCPbt5brKqopZORZOxrqKzMs7m7sas3oevBi7MLfz5XK5vrm5GcYuzO18jDEuLy//PPSFcPq81CUjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiNzPsYYu93ut0NfCKfvfJqm+9vbW2MXZudYOBmDFzLubGQMXsgYvJAxeCHjpS4ZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGBoGMdRUZdzYy1lVkrKvIWFeR8VKXjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjMELGYMXMo6FkzF4IePORsbghYzBCxmDFzJe6pIRGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZExeCFj8ELGsXAyBi9kPCCQ8TVKxrqKjHUVGesqMh4QyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuNYOBnrKjLubGQMXsgYvJAxeCHjpS4ZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9kHAsnY/BC5uWdjdNxlN9Sz4OXQ18I+3Os46XFcrlcbzab3Wq1ejz0xfDfPTw8fNlut+u7u7uLMcZxxTbGGKvV6vH6+vrroS+Gvbk89AW8xktdMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2Mosxvh9LOfSFsB/H/G+5mKbpfrvdrseRHkvh8451vORY+Gk6ymPhBi9kPCCQERsZsZERGxmxkREbGbGRERsZsZH5C92r59JnnqEkAAAAAElFTkSuQmCC"">",0(0.0%)
3,labels[object],1. O2. B-LOC3. B-PER4. I-PER5. B-ORG6. I-DATE7. B-DATE8. I-ORG9. I-LOC,"34,648 (94.1%)566 (1.5%)522 (1.4%)440 (1.2%)179 (0.5%)170 (0.5%)130 (0.4%)129 (0.4%)21 (0.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAADPCAYAAADifqNtAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAADTklEQVR4nO3cMW4bVxhG0X8IwrYaEgQB

Nous initialisons le modèle de base

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
model = AutoModelForTokenClassification.from_pretrained("masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)


Cette fonction permet de parcourir un string retourner un tableau contenant le début (index) de chaque mot dans le string.
En effet  le pipeline du  modèle "masakhane/afroxlmr-large-ner-masakhaner-1.0_2.0" est basé sur une compréhension sur le début de chaque mots.Ce qui nous amène à prendre en compte ce format pour utiliser ce pipeline de facon efficace.

In [11]:
import re

def get_word_starts(sentence):
    words = re.findall(r'\S+', sentence)
    word_starts = {}
    start = 0

    for word in words:
        word_starts[start] = word
        start += len(word) + 1

    return word_starts

Cette fonction comme son nom l'indique ne fait qu'attribuer une entitée à chaque mots en fonction des performances obtenues sur la pipeline de la prédiction du token passer en entrer.

In [12]:
def assign_entities_to_words(word_dict, entity_list):
    entities = {}

    for word_start, word in word_dict.items():
        entities[word] = "O"

        for entity in entity_list:
            if entity['start'] == word_start:
                entities[word] = entity['entity']
                break

    return entities

Inférence de la prédiction de notre algorithme sur une phrase

In [13]:
assign_entities_to_words(get_word_starts("Ali dem Bu nu delloo woon waat yi farañse àbb seen i woroom , lu farañsey dese ciy waat ? "),nlp("Ali ak dem Dakar ak  Bu nu delloo woon waat yi farañse àbb seen i woroom , lu farañsey dese ciy waat ? "))

{'Ali': 'B-PER',
 'dem': 'O',
 'Bu': 'O',
 'nu': 'B-LOC',
 'delloo': 'O',
 'woon': 'B-PER',
 'waat': 'O',
 'yi': 'O',
 'farañse': 'O',
 'àbb': 'O',
 'seen': 'O',
 'i': 'O',
 'woroom': 'O',
 ',': 'O',
 'lu': 'O',
 'farañsey': 'O',
 'dese': 'O',
 'ciy': 'O',
 '?': 'O'}

Créaction du test sentences

In [14]:
# Aggregate lists of words into sentences test set

test = pd.read_csv("/kaggle/input/nerafrican/wol/test.txt", sep=" ", header=None)
test.columns = ["Word", "Tag"]
test = test.drop("Tag", axis=1)
test_sentences = []
sent = ""
for index, row in test.iterrows():
    sent += str(row.Word) + " "
    if row.Word in [".", "?", "!"]:
        test_sentences.append(sent)
        sent = ""

Nous utilisons ici notre algorithme de base pour faire la prédiction sur le test sentence Listes de prédiction sur le test sentence

In [15]:
# List to store the predictions for all sentences
all_predictions = []


# Iterate through the list of sentences and make predictions
for sentence in test_sentences:

    predictions = assign_entities_to_words(get_word_starts(sentence),nlp(sentence))

    all_predictions.append(predictions)

Voir si il n'y a pas de prédiction manquante

In [16]:
len(all_predictions)

462

In [17]:
len(test_sentences)

462

Extraction de la prédiction pour chaque mot dans un dataframe

In [18]:
word_label_pairs = []

for sentence, labels in zip(test_sentences, all_predictions):
    words = sentence.split()

    for word in words:
        label = labels.get(word, 0)  # Obtenez le label du dictionnaire ou 0 par défaut
        word_label_pairs.append((word, label))

# Créez un DataFrame à partir de la liste de tuples
df = pd.DataFrame(word_label_pairs, columns=["word", "label"])

Formatage au format d'envoie du fichier de submission

In [19]:
df

,word,label
0,Bu,O
1,nu,O
2,delloo,O
3,woon,O
4,waat,O
...,...,...
11677,faatu,O
11678,ngir,O
11679,mu,O
11680,folliku,O


In [20]:
df["id"]    = range(len(df))
df = df.drop(columns='word')


In [21]:
df = df[['id', 'label']]

In [22]:
df.head()

,id,label
0,0,O
1,1,O
2,2,O
3,3,O
4,4,O


Nous obtenons ainsi un premier fichier de submission avec  notre modèle de base et notre algorithme de base

In [23]:
df.to_csv('submission.csv', index=False)
!wc -l submission.csv

11683 submission.csv


In [24]:
from IPython.display import FileLink

FileLink(r'submission.csv')

/kaggle/working/submission.csv

Génerer un téléchagement de fichier de submission automatique sur google colab pour la prédiction du modèle de base sur l'ensemble test

In [25]:
# from google.colab import files
# files.download("submission.csv")

Les performances de ce modèle et de notre algorithme de détection d'entités peuvent etre améliorer avec un fine tuning supplémentaire est notre hypothèse suivante

Nous construisons ici notre propre modèle

Model Building

Nous construisons ici notre modèle personnalisé.

---



In [26]:
# Create the aggregated dataframes
agg_train = pd.DataFrame(zip(train_sentences, train_tags), columns=["Sentence", "labels"])
agg_val = pd.DataFrame(zip(val_sentences, val_tags), columns=["Sentence", "labels"])
# agg_test = pd.DataFrame(zip(val_sentences, val_tags), columns=["Sentence", "labels"])



In [27]:
# Convert sentences and tags into list
train_sentences_list = agg_train['Sentence'].tolist()
train_tags_list = agg_train['labels'].tolist()

val_sentences_list = agg_val['Sentence'].tolist()
val_tags_list = agg_val['labels'].tolist()

In [28]:
# Tokenize both training and validation data
train_encodings = tokenizer(train_sentences, truncation=True, padding="max_length", is_split_into_words=True)
val_encodings = tokenizer(val_sentences, truncation=True, padding="max_length", is_split_into_words=True)

Mapping de tag et id

In [29]:
tag_to_id = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-DATE": 7,
"I-DATE": 8
}


Maping de id et tag

In [30]:
id_to_tag = {v: k for k, v in tag_to_id.items()}

In [31]:
id_to_tag

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-DATE',
 8: 'I-DATE'}

Creaction de data frame pour le train et le val

In [32]:
# Initialize a DataFrame
train_data_set = pd.DataFrame(agg_train)

# Tokenize the sentences
train_data_set['tokens'] = train_data_set['Sentence'].apply(lambda x: x.split())
train_data_set['tokens'] = train_data_set['tokens']

# Extract the NER tags and create a sequence of NER tag IDs
train_data_set['ner_tags'] = train_data_set['labels'].apply(lambda tags: [tag_to_id[tag] for tag in tags])
train_data_set['ner_tags_str'] =train_data_set['labels']

# Drop the 'Sentence' and 'Tags' columns
train_data_set =train_data_set.drop(columns=['Sentence', 'labels'])

# Reset the index to create an 'id' column
train_data_set = train_data_set.reset_index()
train_data_set = train_data_set.rename(columns={'index': 'id'})

In [33]:
# Initialize a DataFrame
val_data_set = pd.DataFrame(agg_val)

val_data_set['tokens'] = val_data_set['Sentence'].apply(lambda x: x.split())
val_data_set['tokens'] = val_data_set['tokens']

val_data_set['ner_tags'] = val_data_set['labels'].apply(lambda tags: [tag_to_id[tag] for tag in tags])
val_data_set['ner_tags_str'] = val_data_set['labels']


val_data_set =val_data_set.drop(columns=['Sentence', 'labels'])

val_data_set = val_data_set.reset_index()
val_data_set = val_data_set.rename(columns={'index': 'id'})


In [34]:
train_data_set


,id,tokens,ner_tags,ner_tags_str
0,0,"[SAFIYETU, BÉEY, Céy, Koronaa, !]","[1, 2, 0, 0, 0]","[B-PER, I-PER, O, O, O]"
1,1,"[Doomu, -, jàngoro, bii, yëngal, na, jamono, .]","[0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O, O]"
2,2,"[Ku, mënoon, a, gëm, lu, ndawe, nii, dina, noo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O, O, O, O, O, O]"
3,3,"[Tax, na, ba, kenn, ñemeetul, a, .]","[0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O]"
4,4,"[SÉEX, TIIJAAN, JÉEY, (, Xeltukat, /, Enda, -,...","[1, 2, 2, 0, 0, 0, 3, 0, 3, 0, 0, 0, 1, 2, 2, ...","[B-PER, I-PER, I-PER, O, O, O, B-ORG, O, B-ORG..."
...,...,...,...,...
1796,1796,"[Ndax, yow, jàpp, nga, ni, bor, bi, dañ, koo, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O, O, O, O, O, O, O]"
1797,1797,"[Askani, Afrig, ,, ëllëg, ,, ëllëgu, booloo, l...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O,..."
1798,1798,"[Ñàkk, naa, nit, ku, ma, bëggoon, bu, baax, ,,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1799,1799,"[Jàpp, naa, ni, waxu, ku, mer, la, ;, xéy, -, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [35]:
val_data_set


,id,tokens,ner_tags,ner_tags_str
0,0,"[Ndekete, ,, da, cee, am, ñu, xër, ci, diine, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,1,"[Gisaanekat, yi, tamit, kenn, demul, ñu, des, .]","[0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O, O]"
2,2,"[Ñii, a, ngi, naan, koronaawiris, bi, amul, ,,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,3,"[Moo, tax, ,, bi, leen, jawriñ, ji, kalaamee, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER,..."
4,4,"[Waaye, ,, bi, leen, DIC, wooloo, ,, ku, nekk,...","[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O]"
...,...,...,...,...
254,254,"[Yenn, làkki, islaaf, ak, yu, indoy, amerig, :...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O,..."
255,255,"[Wolof, :, àbb, na, ko, ay, waat, yu, bari, li...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
256,256,"[🤩, Niral, :, laptot, :, lapptoo, ,, banane, :...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
257,257,"[), 😌, Ci, gàttal, ,, wenn, làkk, setul, ci, à...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


Créaction de dictionnaire de donnée pour le train et le val conforme au format CoNLL

In [36]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_dict(train_data_set)  # Convert  training data to a Dataset
val_dataset = Dataset.from_dict(val_data_set)      # Convert  validation data to a Dataset

# Create a DatasetDict
data_dict = DatasetDict({'train': train_dataset, 'val': val_dataset})



In [37]:
data_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 1801
    })
    val: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'ner_tags_str'],
        num_rows: 259
    })
})



# 1.   Tokenization



In [38]:
tokenizer.is_fast

True

In [39]:
train_data_dict = train_data_set.to_dict(orient='records')


In [40]:
val_data_dict = val_data_set.to_dict(orient='records')


In [41]:
train_data_set['tokens'][0]

['SAFIYETU', 'BÉEY', 'Céy', 'Koronaa', '!']

In [42]:
print(tokenizer("ali dem Dakar").tokens())

['<s>', '▁ali', '▁dem', '▁Da', 'kar', '</s>']


In [43]:
inputs = train_data_set['tokens'][0]
inputs = tokenizer( train_data_set['tokens'][0], is_split_into_words=True)

print(inputs.tokens())

['<s>', '▁', 'SAF', 'IYE', 'TU', '▁B', 'É', 'EY', '▁Cé', 'y', '▁Koro', 'na', 'a', '▁!', '</s>']


In [44]:
print(train_data_set['tokens'][0])
print(train_data_set['ner_tags_str'][0])

['SAFIYETU', 'BÉEY', 'Céy', 'Koronaa', '!']
['B-PER', 'I-PER', 'O', 'O', 'O']


In [45]:
train_data_set['tokens'][0]

['SAFIYETU', 'BÉEY', 'Céy', 'Koronaa', '!']

In [46]:
inputs.word_ids()


[None, 0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 3, 4, None]

In [47]:
def align_labels_with_tokens(labels, word_ids):
    aligned_labels = []
    for word_id in word_ids:
        if word_id is None:
            aligned_labels.append(-100)
        else:
            aligned_labels.append(labels[word_id])
    return aligned_labels

In [48]:
train_data_dict[0]

{'id': 0,
 'tokens': ['SAFIYETU', 'BÉEY', 'Céy', 'Koronaa', '!'],
 'ner_tags': [1, 2, 0, 0, 0],
 'ner_tags_str': ['B-PER', 'I-PER', 'O', 'O', 'O']}

In [49]:
labels = [tag_to_id[tag] for tag in train_data_dict[0]['ner_tags_str']]
word_ids = inputs.word_ids()
print(labels, train_data_dict[0]['ner_tags'])

[1, 2, 0, 0, 0] [1, 2, 0, 0, 0]


In [50]:
align_labels_with_tokens(labels, word_ids)


[-100, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, -100]

In [51]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  all_labels = examples['ner_tags']

  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels, word_ids))

  tokenized_inputs['labels'] = new_labels

  return tokenized_inputs

Tokenization for all datasets

In [52]:
tokenized_datasets = data_dict.map(tokenize_and_align_labels, batched=True,remove_columns=data_dict['train'].column_names )


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Data Collation and Metrics

In [53]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1801
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 259
    })
})

In [54]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [55]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[     0,      6, 176846, 212129,  19454,    335,   6642,  53253,  53285,
             53, 105622,     76,     11,    711,      2,      1,      1,      1,
              1,      1],
        [     0,    984,    306,     34,     20,   1647,   1298,    449,  12393,
            333,     14,    113,   2279,   2870,     24,     79,  64176,      6,
              5,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[-100,    1,    1,    1,    1,    2,    2,    2,    0,    0,    0,    0,
            0,    0, -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0, -100]])}

In [56]:
!pip install seqeval
!pip install evaluate

import evaluate
metric = evaluate.load('seqeval')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [57]:
data_dict['train'].features

{'id': Value(dtype='int64', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'ner_tags_str': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [58]:
label_names = [
    'O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'
]

In [59]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [60]:
labels=train_data_set['ner_tags'][0]
labels = [label_names[i] for i in labels]
labels


['B-PER', 'I-PER', 'O', 'O', 'O']

In [61]:
predictions = labels.copy()
#Nous changeons volontairement la valeur du résultat attendu pour voir son impacte sur nos metric ./Quand on efface l'error creer notre metric est performant . Ce qui prouve que notre métric est pret.
predictions[2] = "B-PER"


Test de métric sur un simple exemple.Cela garentit que notre métrique est prèt pour utilisation car nous avions modifier une valeur juste et son impacte est réflécter avec un pourcentage équivalent

In [62]:
metric.compute(predictions=[predictions], references=[labels])


{'PER': {'precision': 0.5,
  'recall': 1.0,
  'f1': 0.6666666666666666,
  'number': 1},
 'overall_precision': 0.5,
 'overall_recall': 1.0,
 'overall_f1': 0.6666666666666666,
 'overall_accuracy': 0.8}

# Mise en place de metric pour tous notre data set

In [63]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

In [64]:
id_to_tag

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-DATE',
 8: 'I-DATE'}

In [65]:
tag_to_id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-DATE': 7,
 'I-DATE': 8}

In [66]:
model.config.num_labels


9

Avec 1 epoc la performance de notre modèle est plutot mieux

Pour des raisions d'espaces nous prenons juste 2 epocs . Mais sur colab on va jusqu'à 5 et plus avec un temps de 10 min maxi pour 5 epocs.

In [67]:
from transformers import TrainingArguments

args = TrainingArguments(    output_dir = "bylyai",
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=2,
                         weight_decay=0.01)

# Définission des paramètres

In [68]:
from transformers import Trainer
trainer = Trainer(model=model,
                  args=args,
                  train_dataset = tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['val'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)



# Entrainement du modèle

Sur kaggle le trainer.train() indique un code sur wandb pour le ampper sur hugging face dans le but de suivre notre entrainement. Inscrivez vous juste wandb pour avoir un code.L'avantage est que le code permet de suivre l'entrainement sur un graphique sur huggingface.Sur colab cela n'exige pas de code .Donc toute adaptation des routes (path des fichiers data) sur colab n'exige aucune inscription lors de la compilation du code.

In [69]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.083613,0.883333,0.854839,0.868852,0.991078
2,No log,0.085181,0.844000,0.850806,0.847390,0.989294


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=226, training_loss=0.059744210369819035, metrics={'train_runtime': 1105.5674, 'train_samples_per_second': 3.258, 'train_steps_per_second': 0.204, 'total_flos': 572448986011884.0, 'train_loss': 0.059744210369819035, 'epoch': 2.0})

In [70]:
# trainer.save_model("bylyai")

In [71]:
from transformers import pipeline

checkpoint = "/kaggle/working/bylyai/checkpoint-226"

In [72]:
token_classifier = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)


Notre modèle entrainer prédire très bien les entités dans une phrase. Mais l'étiquetage de ces derniers n'est pas cohérent.La cohérence est retrouvé en lisant comme ceci: x->y signifie que pour x prédire, l'étiquetage réel est y. Voici la bonne étiquettage de notre modèle d'après nos tests:

  

*    Date->Personne
*   LOC->Date
*   ORG->LOC
*   PER->ORG





Une correction de l'alignement des tokens peut aussi résoudre la permuttation lors de notre tokenization mais nous le gérons avec une fonction custom

Dans ce exemple, la séquence Li est identifié comme une ORG d'après notre algo cela veut dire LOC .Meme logique pour les autres

In [73]:
token_classifier("Liwisiyaan, Niiw Orleyoon, mel na ne tawat ji nit ñu ñuul ñi la singali nde ñoom 33 % mooy seen ub lim waaye 70 % ca ña faatu ay nit ñu ñuul la ñu.")


[{'entity_group': 'ORG',
  'score': 0.98591864,
  'word': 'Li',
  'start': 0,
  'end': 2},
 {'entity_group': 'ORG',
  'score': 0.96464336,
  'word': 'wis',
  'start': 2,
  'end': 5},
 {'entity_group': 'ORG',
  'score': 0.5317498,
  'word': 'iyaan',
  'start': 5,
  'end': 10},
 {'entity_group': 'DATE',
  'score': 0.9948224,
  'word': 'Nii',
  'start': 12,
  'end': 15},
 {'entity_group': 'DATE',
  'score': 0.9987314,
  'word': 'w Orleyoon',
  'start': 15,
  'end': 25}]

In [74]:
import re

def assign_entities_IOB(phrase, entities):
    # Diviser la phrase en mots en utilisant des espaces comme séparateurs et supprimer les espaces autour de chaque mot
    mots = [mot.strip() for mot in phrase.split()]

    resultat = []

    for mot in mots:
        if len(mot) == 0:
            continue

        entity = "O"

        # Vérifier si le mot est alphanumérique
        if mot.isalnum():
            for entite in entities:
                start_index = phrase.find(mot)
                end_index = start_index + len(mot) - 1

                if entite['start'] <= start_index <= entite['end'] or entite['start'] <= end_index <= entite['end']:
                    if start_index == entite['start']:
                        entity = "B-" + entite['entity_group']
                    else:
                        entity = "I-" + entite['entity_group']
                    break

        resultat.append((entity, mot))

    return transform_tags(resultat)


def transform_tags(tag_list):
    result = []
    prev_tag = None

    for tag, word in tag_list:
        if prev_tag == 'O' and tag.startswith('I-'):
            tag = 'B' + tag[1:]
        result.append((tag, word))
        prev_tag = tag

    return result


In [75]:
def exchange_labels(dataframe):
    # Créer un dictionnaire de correspondance pour les échanges de labels
    label_mapping = {
        'B-DATE': 'B-PER',
        'B-LOC': 'B-DATE',
        'B-ORG': 'B-LOC',
        'B-PER': 'B-ORG',
        'I-DATE': 'I-PER',
        'I-LOC': 'I-DATE',
        'I-ORG': 'I-LOC',
        'I-PER': 'I-ORG'
    }

    # Appliquer le remplacement en utilisant le dictionnaire de correspondance
    dataframe['label'] = dataframe['label'].map(label_mapping).fillna(dataframe['label'])

    return dataframe

In [78]:
# List to store the predictions for all sentences
all_predictions_new = pd.DataFrame(columns=['label', 'Word'])

# Iterate through the list of sentences and make predictions
for sentence in test_sentences:

    predictions =exchange_labels(pd.DataFrame(assign_entities_IOB(sentence,token_classifier(sentence)), columns=['label', 'Word']))

    all_predictions_new = pd.concat([all_predictions_new, predictions], ignore_index=True)


In [79]:
all_predictions_new

,label,Word
0,O,Bu
1,O,nu
2,O,delloo
3,O,woon
4,O,waat
...,...,...
11677,O,faatu
11678,O,ngir
11679,O,mu
11680,O,folliku


In [80]:
all_predictions_new["id"]    = range(len(df))
all_predictions_new = all_predictions_new.drop(columns='Word')


In [81]:
all_predictions_new= all_predictions_new[['id', 'label']]
all_predictions_new


,id,label
0,0,O
1,1,O
2,2,O
3,3,O
4,4,O
...,...,...
11677,11677,O
11678,11678,O
11679,11679,O
11680,11680,O


In [82]:
all_predictions_new.to_csv('submission_custom.csv', index=False)
!wc -l submission.csv

11683 submission.csv


In [83]:
from IPython.display import FileLink

FileLink(r'submission_custom.csv')

/kaggle/working/submission_custom.csv

Enregistrement d'une sauvegarde du modèle pour la mise en place de notre api de détection de NER

In [ ]:
!zip -r bylyai_afroml_fine_ner.zip "bylyai"
